https://youtu.be/wEoyxE0GP2M
http://d2l.ai/index.html

## Activation functions

### *Sigmoid function*
* Map tous ses *input* dans $[0, 1]$
* A une bonne interprétabilité en termes de saturation / activation d'un neurone.
Problèmes: 
* Gradient $\sigma'$ très faible / quasi nul pour les grandes valeurs en valeur absolue -> pose problème pour la *back-propagation* (*vanishing gradient*).
* Outputs non-centrés en zéro et positif, or $\frac{\partial sigma}{\partial w} = x.\sigma'(w^{\top}.x +b)$. Cet élément apparaissant dans la *chain rule* dans la *back-propagation* est donc toujours du même signe, ce qui nous contraint dans les directions admissibles lors de la descente de gradient.
* L'exponentielle de la fonction est relativement coûteuse en termes de calcul.

### *tanh*
Map tous ses *inputs* dans $[-1, 1]$ qui sont donc centrés sur zéro contrairement à la *sigmoid*. En revanche, comme il s'agit d'une sigmoid *rescaled* et transaltée ($tanh(x)=2\sigma(2x)-1$), on conserve les autres problèmes de la *sigmoid*.

### ReLU (*Rectified Linear Unit*)
La fonction ReLU est $x \mapsto max(0,x)$. Avantages: elle ne sature pas dans le domaine positif, n'a pas les problèmes de gradients de la *sigmoid* et de tanh et les calculs aussi bien en *forward* qu'en *backward-propagation* sont très efficace. 

Inconvénients: 
* Les *outputs* ne sont pas centrés en zéro, 
* La fonction sature dans le domaine négatif (à zéro -> "*ReLU dies*")avec le problème associé de gradients nuls. Notamment, tous les $x$ pour lesquels $w^{\top}.x +b \leq 0$ ne donnent lieu à aucune activation de la cellule et ne contribuent pas à l'apprentissage (ex: si on a pas eu de chance dans l'*update* / l'initialisation nos poids $w$ et $b$). Cela pousse par exemple certains à initialiser positivement leurs biais (au lieu de les initialiser à zéro), le bénéfice systématique de la mesure n'étant pas consensuel.

La *leaky* ReLu ($x \mapsto max(0,01x, x)$) ne saturant plus dans le domaine négatif, les problèmes associés disparaissent, les principaux avantages de la ReLU étant conservés.

Variante: Parametrix ReLU (PReLU): $x \mapsto max(\alpha.x, x)$ où $\alpha$ est appris.

Généralisation: *Maxout* $x \mapsto max(w_{1}^{\top}.x + b_1, w_{2}^{\top}.x +b_2)$ A tous les avantages de l'efficacité de la linéarité, de l'absence de saturation (gradients nuls) et de la saturation à zéro sur une partie du domaine ("die") mais au pris de doubler le nombre de paramètres à apprendre par unité.

En pratique: commencer avec des ReLU, essayer ses variantes (*leaky* ReLU, maxout, etc.), éventuellement essayer $tanh$ mais ne pas en attendre des miracles, ne pas utiliser la sigmoïde.

## Preprocessing
Images: on centre (comme toujours, pour éviter les biais, que des valeurs soient par exemple toujours positives ou négatives) mais en général on ne réduit pas.

Remarque: On a vu (notamment pour la sigmoïde) qu'il est souhaitable que nos valeurs d'entrée soient autour de zéro (on est alors pour la sigmoïde dans le domaine où elle ne sature pas). Le preprocessing des données résout ce problème (le centrage) mais seulement pour la première couche. Mais c'est un problème (le centrage des données en entrée de couche) qu'on retrouve d'autant plus souvent que nos réseaux sont profonds.

## Initialisation des poids
Problème d'une initialisation homogène (ex: 0): tous les neurones vont produire le même *output*, donc leurs gradients (de la *loss* par rapport à leur activation) seront tous les mêmes, donc leurs poids seront updatés de la même manière. On aura donc des couches au sein desquelles les neurones seront identiques. On sera donc dans une situation où on se prive que chaque neurone se focalise sur une *feature* particulière des données et donc d'un modèle d'ensemble plus riche et qui généralise mieux. Il faut une initialisation qui brise les symétries.

Une initialisation homogène avec de faibles valeurs aléatoire (proches de zéro) peut ne pas fonctionner non plus: on va dès les premières couches avoir des activation en moyenne nulles ($w^{\top}.x$ sera proche de zéro car $w$ très petit) et des gradients tout aussi faibles (ne pas oublier qu'avec la *chain rule*, on multiplie par l'*input* de la couche courante mais aussi les gradients d'entrée par les coefficients de la couche $l+1$).

Une initialisation avec de grandes valeurs aléatoires n'est pas mieux avec des fonctions d'activation qui saturent. Par exemple avec $tanh$, les distributions des sorties sont rapidement des bimodales très ressérées sur $-1$ et $1$ avec des gradients nuls.

Exemple d'initialisation classique: Xavier initialisation: Chaque couche est initialisée de façon différence: les valeurs sont tirées d'une gaussienne de variance en $1/n_{input}$. Marche bien quand / fait l'hypothèse d'une activation linéaire (ex: on est dans la région active d'une $tanh$ / sigmoïde). Quand on passe à une ReLU cependant, la distribution des activations pique à 0 et la situation empire à mesure qu'on s'enfonce dans le réseau. Il a été montré que simplement prendre une variance de $1/(2.n_{input})$ résout le problème, les distributions d'activation sont alors beaucoup plus stables d'une couche à l'autre et c'est ce qu'on recherche.

## Batch normalization
https://towardsdatascience.com/batch-normalization-in-3-levels-of-understanding-14c2da90a338#b93c

Ces problèmes de stabilité de distribution des activations (ne pas avoir des distributions qui tendent au fur et à mesure vers des pic ressérés sur une valeur) est un peu le pendant côté *forward-propagation* des *vanishing*/*exploding* gradients côté *back-propagation*.

L'idée de la *batch normalization* est de centrer-réduire les *inputs* (avec la moyenne et la variance empirique de la *batch* entière) d'une couche donnée avant leur passage dans la fonction d'activation. Mais ce n'est pas tout, les *inputs* normalisés $\hat{x}$ sont ensuite *reshifted* et *rescaled* comme suit: $\hat{x} \leftarrow \gamma.\hat{x} + \beta$. Les paramètres $\beta$ et $\gamma$ sont en revanche appris: on laisse au réseau la flexibilité de trouver la meilleure normalisation. La *batch normalization* est ainsi une couche supplémentaire qui s'insère avant chaque couche "classique".

**Remarques**:
* L'étape de *shift* and *rescale* est peut être d'autant plus nécessaire que dans le cas où on travaille avec des fonction de type sigmoïde, il n'est pas souhaitable que l'ensemble du *dataset* soit maintenu dans leur zone d'approximation linéaire: on ne profite alors pas des non linéarités de la fonction d'activation. D'un autre côté on ne souhaite pas que l'activation sature sur une trop grande part du *dataset*, on laisse donc le réseau choisir quelle est finalement la meilleure normalisation à effectuer avec chaque couche.
* En mode "test", la normalisation n'est pas faite sur la batch comme en *training* mais à l'aide d'une moyenne et d'une variance empirique apprise sur le *training set* et conservée dans ce but.

## Optimisation
Un des enjeux de l'optimisation est qu'elle ne se fasse pas piéger dans des minimas locaux de la *loss* ou au niveau de *saddle points* de celle-ci (au voisinage desquels le gradient est très faible et sur lesquels il est nul). En haute dimension, les seconds sont davantage problématiques que les premiers qui finalement se recontrent de moins en moins. En effet, un minimum local se caractérise par le fait que la *loss* augmente dans toutes les directions de son voisinage, ce qui est d'autant moins probable quand le nombre de dimensions est élevé. On aura vraisemblablement au moins une direction dans laquelle la *loss* décroit i.e. on a affaire à un *saddle point*.

*Stochastic gradient descent* (SGD): on estime la *loss* et son gradient à partir de d'une *mini-batch* -> *noisy estimates*

Classic SGD: $w_{n+1} \leftarrow w_n - \alpha.\nabla f(w_n)$

Une première amélioration est l'introduction d'un terme d'inertie (*momentum*): la direction dans laquelle on se déplace à l'étape $n$ est une moyenne pondérée (lissage exponentiel) des gradients calculés lors de toutes les étapes antérieures. Cela s'écrit simplement:

$$w_{n+1} \leftarrow w_n - \alpha.v_{n+1}$$
$$v_{n+1} \leftarrow \rho.v_{n} + \nabla f(w_{n+1})$$

Le procédé introduit un nouvel hyperparamètre $\rho$ typiquement compris entre 0.9 et 0.99. On voit facilement que:

$$v_n = \nabla f(w_n) + \rho.\nabla f(w_{n-1}) + \rho^2.\nabla f(w_{n-2}) + \dots + \rho^k.\nabla f(w_{n-k}) + \dots $$

L'ajout de cette inertie a notamment pour avantage:
* De lisser le bruit inhérent à la SGD
* Dans le cas mal définis où la *loss* varie beaucoup plus vite dans une direction que dans une autre, permet d'éviter les zigzags autour de la direction suivant laquelle la *loss* varie lentement.
* Dans une zone où le gradient diminue fortement, notre déplacement ne diminue pas aussi vite grace à la mémoire des gradients antérieurs.

Variante Nesterov: Le gradient pour passer de l'étape $n$ à $n+1$ n'est pas calculé en $w_n$ mais en $w_n+\rho.v_{n}$:

$$w_{n+1} \leftarrow w_n + v_{n+1}$$
$$v_{n+1} \leftarrow \rho.v_{n} + - \alpha.\nabla f(w_n+\rho.v_{n})$$

Remarques: 
* $v_0=0$
* Gradient et loss ne sont plus évalués au même point. Ce problème peut être résolu par un changement de variables $\tilde{w}_n = w_n + \rho.v_n$. Les équations se transforment pour se rapprocher du cas d'inertie simple: 

$$v_{n+1} \leftarrow \rho.v_{n} + - \alpha.\nabla f(\tilde{w}_n)$$
$$\tilde{w}_{n+1} \leftarrow \tilde{w}_n+ v_{n+1} + \rho.(v_{n+1}-v_{n})$$

Nesterov peut alors se voir comme un cas où le déplacement se fait dans la direction du gradient avec inertie avec une correction supplémentaire correspondant à l'écart entre les deux dernières directions.

## Learning rates
Au delà d'avoir le bon ordre de grandeur pour converger efficacement, plusieurs techniques existent pour faire décroitre le *learning rate* au cours de l'apprentissage:
* *Step decay*: on fait décroitre le *learning rate* périodiquement, toutes les $n$ époques.
* *Exponential decay*: $\alpha = \alpha_0.e^{-k.t}$
* *1/t decay*: $\alpha = \frac{\alpha_0}{1+k.t}$

## Régularisation
Les techniques classiques de régularisation (ajouter un terme de régularisation L1/L2 à sa *loss function*) ne sont pas forcément les plus adaptées au *deep learning*. 

Drop out: on va pour toutes ou partie des couches du réseau, forcer leur activation à zéro lors de la *forward propagation* (le reste des étapes, *back-propagation* et *weight update*, étant faites normalement). Les cellules inactivées sont choisies aléatoirement, la propabilité pouvant ne pas être choisie identique pour toutes les couches. En général se sont les couches denses qui font prioritairement l'objet de cette régularisation.

Une interprétation de pouquoi cela fonctionne est qu'on force le réseau à ne pas développer de *features* spécialisées mais au contraire d'avoir des features qui combinent différents aspects, ce qui l'aide à mieux généraliser: quand on lui enlève une unité qui aurait pu apprendre une feature spécialisée, on perd moins en performance si les unités restantes on incorporé celle-ci à leur apprentissage.

L'utilisation du drop out exige le caractère aléatoire introduit par la technique soit moyenné. Une façon simple et fonctionnelle appelée inverted drop out consiste à rescale les activations soit au training time par la probabilité $1/p$ soit au test time en multipliant les activation de la couche à laquelle le drop out a été appliqué au training par $p$.

Toutes les techniques de régularisation (drop out, batch norm, data augmentation) ont en commun d'ajouter de la stochasticité au training time: toutes les batch ne voient pas le même drop out mask, la même normalisation, la data augmentation donnent différentes versions d'un même label, etc, qui est moyenné (au moins approximativement) au test time: les activations sont rescaled pour le drop out, les couches batch norm utilisent des estimations globales pour standardiser, en data augmentation on utilise des crops fixes au test time (et non aléatoires). Cette stochasticité est porteuse d'un effet régularisant. 

D'autres techniques l'exploitent: drop connect (on force des coefficients à zéro), drop aléatoirement des layers, etc.

Transfer learning: d'autant plus favorable si notre dataset est proche de celui sur lequel le modèle a été entrainé. Si on a peu de données on peut se contenter de réentrainer la/les dernières couches (les plus spécialisées) du réseau. Si on a plus de données on peut aller jusqu'à en réentrainer d'autres. On utilise en général un learning rate plus faible que celui utilisé le premier entrainement du réseau (1/10 est un bon départ). 

## Preprocessing de texte
On considère généralement trois grandes catégories de preprocessing de texte:
* *Denoising*
* *Text tokenization*
* *Text normalization*

Le *denoising* est en général très spécifique au *use case*/à la source de donnée et peut très bien se définir par complémentarité par rapport aux deux autres. Il s'agit par exemple d'opérations de nettoyage et de parsing propre au format d'entrée des données (HTML, XML, JSON, etc.).

On peut y inclure des étapes pouvant faciliter la *tokenization* comme l'expansion ("didn't" => "did not").

La *tokenization* qui consiste à diviser le texte en plus petites entités appelées *tokens*: paragraphes, phrases ou mots suivant les *use cases*. Le terme est parfois réservé à la subdivision en mots, la division en paragraphes ou phrases étant désignée sous le terme de *segmentation*.

Cette étape est faussement simple. Par exemple:
* Subdivision en mots à l'aide de splits sur une liste de caractères. On prend l'apostrophe: "Hawai'i", "didn't" 
* Subdivision en phrases à l'aide des signes de ponctuation: "What is all the fuss about?" asked Mr. Peters.

Une bonne *tokenization* n'a donc rien d'évident et mieux vaut se reposer sur des *third-party packages*.

La *normalization* qui venant généralement après la *tokenization*, consiste à mettre rendre identiques des *tokens* sémantiquement proches (ce qui au passage réduit la taille du vocabulaire sur lequel on va travailler). On la subdivise généralement en trois grand type de transformations: 
* La *stemming* : Elle consiste à retirer à un mot ses préfixes, suffixes, infixes, etc. pour ne conserver que la racine, le radical (*stem*). Ex: "running" => "run", "fishes => fish".
* La *lemmatization*: Elle consiste à déterminer pour chaque mot, le mot dont il est dérivé au sens large (le *lemma*). Ex: "better" => "good"
* Les autres opérations qui peuvent varier suivant les *use cases* et parmis lesquelles on trouve: 
    * Le remplacement des caractères "spéciaux" (ex: non-ASCII). 
    * Passer tous les mots en *lower case*. 
    * Supprimer la ponctuation, les *white spaces*.
    * Supprimer les *stop words*. Les *stop words* sont un ensemble de mots très répendus dans le language et le plus souvent peu porteurs d'information (ex: les déterminants, les pronoms, auxiliaires être et avoir, etc.) et donc souvent supprimés (*stopped* = *filtered out*). Il n'y a généralement pas d'accord sur la liste des *stopped words* d'un language donné, en particulier entre les différents outils de préprocessing de texte, d'autant que cette liste est facilement dépendante du *use case*.
    * Remplacer les nombres par leur équivalent en texte. 
    * Etc.
    
Remarque: *Stemming* et *lemmatization* sont proches (ils peuvent retourner le même résultat pour un *token* donné, ex: "walking" => "walk") et par certains aspects complémentaires (l'un - en général la *lemmatization* - capture des relations manquées par l'autre, ex: "better" => "good") voir contradictoires (les deux ne retournent pas le même résultat pour un *token* donné). La *lemmatization* est plus puissante mais plus coûteuse que le *stemming* dans le sens où elle cherche à retourner le *lemma* correct en tenant compte du contexte (le *stemming* agit mot par mot et n'est pas *context-aware*). Par exemple, pour le *token* "meeting" employé dans "in our last meeting" et "we are meeting tomorrow", une bonne *lemmatization* retournera un nom dans le premier cas, un verbe dans le second.

Une possible dernière étape consisterait à donner aux *tokens* obtenus une représentation numérique assimilable par un modèle à l'aide de *pretrained embeddings* si on souhaite justement pas apprendre cette représentation.

### Sequence to sequence models

Mécanismes d'attention (*attention mechanisms*)

Les mécanismes d'attention sont une solution au problème d'*information bottleneck* apparaissant dans l'architecture encoder/decoder. Il est en effet attendu de l'état caché en sortie d'encoder qu'il résume à lui seul l'ensemble de l'information de la séquence d'entrée. Cette information étant ensuite la seule base sur laquelle la génération de séquence par le decoder va s'appuyer.

Moyen de donner accès à davantage d'information sur la séquence d'entrée tout en réalisant la "traduction"/génération: à l'étape $i$ de la génération, on va en plus se baser sur les représentations de la séquence d'entrée les plus informatifs pour l'étape courante. D'autant plus utile qu'à l'étape $i$ de la génération, on n'a plus accès directement à l'output de l'encoder mais à l'état caché du decoder à l'étape précédente. Le mécanisme d'attention permet à chaque étape de la génération d'avoir 1) acccès à la séquence d'entrée 2) accès à une représentation informative de la séquence d'entrée pour l'étape courante. A chaque étape de la génération, on rejette un oeil à l'ensemble de la séquence d'entrée et n'en conserve que les représentations (on focalise notre attention) que sur les celle les plus informatives, les plus similaires à l'état caché à l'étape $i$.

On se sert des états cachés de l'encoder comme d'une mémoire. Du point de vue du premier step du decoder par exemple, ce denier n'a plus seulement accès à une représentation unique de la séquence d'entrée correspondant au dernier état caché de l'encoder mais à l'ensemble de ses états cachés.


On peut résumer le mécanisme d'attention comme suit: à chaque étape (*step*) du decoder, on va établir une connection directe à l'encoder pour se focaliser sur une partie particulière de la séquence d'entrée (un ou plusieurs mots, pas forcément consécutifs).

A haut niveau, l'implémentation d'un mécanisme d'attention consiste pour une étape $i$ du decoder, à:
* Calculer le produit scalaire (mesure de similarité) de l'état caché du decoder avec chacun des états cachés de l'encoder (rappel: ils sont de même dimension), on obtient un vecteur de la taille de la séquence d'entrée.
* Ce vecteur est normalisé par passage dans une *softmax* layer pour obtenir une distribution.
* On créé un nouveau vecteur dit *attention vector* (pour l'étape $i$) correspondant à la moyenne des états cachés de l'encoder, pondérée par les poids calculés à l'étape précédente. 
* La prédiction du décoder se base sur la concaténation de ce vecteur d'attention et de l'état caché du decoder à l'étape $i$ (et pas seulement sur l'état caché du decoder). 

Remarque: Traditionnellement, l'input de l'étape $i+1$ du decoder est constitué de la prédiction et de l'état caché de l'étape $i$. Il arrive qu'à la place de l'état caché, on passe la concaténation de l'état caché et du vecteur d'attention pour l'étape $i$.

On peut voir plus généralement le mécanisme d'attention comme une requête (*query*): on utilise la représentation d'un mot (dans le cas encoder/decoder, d'une séquence) comme *query* pour accéder et incorporer l'information d'un ensemble de valeurs (dans le cas encoder/decoder, les état cachés de l'encoder).